In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
model_name = ["resnet34", "mobile_net_v3_large", "efficient_net_b1"]
num_epochs = "100"
lr_start = "0.01"
lr_end = "0.001"
batch_size = "32"

In [ ]:
model_params = {}

In [ ]:
def get_metrics(models, dataset_name, num_epochs, batch_size,
                lr_start, lr_end):
    val_acc = {}
    val_loss = {}
    val_true = {}
    val_pred = {}

    train_acc = {}
    train_loss = {}
    train_true = {}
    train_pred = {}
    
    
    for model_name in models:
        
        with open(f'../../models/val_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'rb') as handle:
            val_history = pickle.load(handle)

        with open(f'../../models/train_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'rb') as handle:
            train_history = pickle.load(handle)
        
        try:
            val_acc[model_name] = [i.item() for i in val_history['acc']]
        except:
            val_acc[model_name] = val_history['acc']
        val_loss[model_name] = val_history['loss']
        val_pred[model_name] = val_history['outputs']
        val_true[model_name] = val_history['targets']

        train_acc[model_name] = train_history['acc']
        train_loss[model_name] = train_history['loss']
        train_pred[model_name] = train_history['outputs']
        train_true[model_name] = train_history['targets']
        
    
    
    return {'train_acc': train_acc, 'train_loss': train_loss, 'train_pred': train_pred, 'train_true': train_true,
                                 'val_acc': val_acc, 'val_loss': val_loss, 'val_pred': val_pred, 'val_true': val_true}

In [ ]:
dataset_name = "ads"
model_params[dataset_name] = get_metrics(model_name, dataset_name, num_epochs, batch_size, lr_start, lr_end)
model_params[dataset_name]['train_acc'].keys()

In [ ]:
colors_acc = {'chest_xray_8': 'yellowgreen', 'covid_pneumonia': 'lightcoral', 'pneumonia': 'cornflowerblue'}
colors_loss = {'chest_xray_8': 'green', 'covid_pneumonia': 'firebrick', 'pneumonia': 'blue'}
middle_flag = {'resnet34': False, 'efficient_net_b1': True, 'mobile_net_v3_large': False}
start_flag = {'resnet34': True, 'efficient_net_b1': False, 'mobile_net_v3_large': False}
end_flag = {'resnet34': False, 'efficient_net_b1': False, 'mobile_net_v3_large': True}

model_names = {'resnet34': "ResNet 34", 
               'mobile_net_v3_large': "MobileNet V3 Large", 
               'efficient_net_b1': "EfficientNet B1"}
dataset_names = {'chest_xray_8': "Chest X-ray 8", 
               'covid_pneumonia': "COVID + Pneumonia", 
               'pneumonia': "Pneumonia"}

In [ ]:

model = model_name[0]
model_name = ["resnet34", "efficient_net_b1", "mobile_net_v3_large"]

In [ ]:
#fig, ax1 = plt.subplots(1,3, figsize=(12, 5))
fig = plt.figure(figsize=(12, 8))
gs = fig.add_gridspec(2, 3, hspace=0, wspace=0)
ax1, ax2 = gs.subplots(sharex='col', sharey='row')


#ax2 = []
fig.suptitle(f'Train & Validation Dataset - F1 & Loss Plots', fontsize=16)
epochs = [i for i in range(int(num_epochs))]
for i in range(3):
    model = model_name[i]
    ax1[i].set_title(f'{model_names[model]}')
    # plt.figure()
    if start_flag[model]:
        ax1[i].set_ylabel('F1 Score')
    if middle_flag[model]:
        ax2[i].set_xlabel('Epoch')
    
    color = 'black'

    for data in model_params.keys():
        ax1[i].plot(epochs, 
                 [f1_score(true, pred, average='macro') for true, pred in zip(model_params[data]['val_true'][model], model_params[data]['val_pred'][model])],
                 label=f'{dataset_names[data]} - Validation F1', 
                 color=colors_acc[data])
        ax1[i].plot(epochs, 
                 [f1_score(true, pred, average='macro') for true, pred in zip(model_params[data]['train_true'][model], model_params[data]['train_pred'][model])],
                 label=f'{dataset_names[data]} - Training F1', 
                 color=colors_acc[data], linestyle='--')

    ax1[i].tick_params(axis='y', labelcolor=color)   
    ax1[i].set_ylim([0, 1.05])

    #ax2.append(ax1[i].twinx())
    color = 'black'
    
    if start_flag[model]:
        ax2[i].set_ylabel('Loss (Log Scale)', color=color)

    for data in model_params.keys():
        ax2[i].plot(epochs, 
                 np.log(model_params[data]['val_loss'][model]),
                 label=f'{dataset_names[data]} - Validation Loss', 
                 color=colors_loss[data])
        ax2[i].plot(epochs, 
                 np.log(model_params[data]['train_loss'][model]),
                 label=f'{dataset_names[data]} - Training Loss', 
                 color=colors_loss[data], linestyle='--')

    ax2[i].set_ylim([-10, 2.5])
    ax2[i].tick_params(axis='y', labelcolor=color)    
    
    if middle_flag[model]:
        ax1[i].legend(bbox_to_anchor=(1., -2.1, 0.41, 0.94))
        ax2[i].legend(bbox_to_anchor=(0., -1.1, 0.41, 0.94))

fig.tight_layout()
#fig.savefig("f1_loss_separate.png", format="png", dpi=600, bbox_inches="tight")
#fig.savefig("f1_loss_separate.eps", format="eps", dpi=1200, bbox_inches="tight", transparent=True)
#fig.savefig("f1_loss_separate.pdf", format="pdf", dpi=1200, bbox_inches="tight", transparent=True)

#fig.legend(bbox_to_anchor=(0., -1., 0.41, 0.94))